In [1]:
CLONE_URL = f"https://github.com/LucaBrembilla/RecSys_Course_AT_PoliMi.git"
get_ipython().system(f"git clone {CLONE_URL}")

import sys
sys.path.append("RecSys_Course_AT_PoliMi")

fatal: destination path 'RecSys_Course_AT_PoliMi' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append("RecSys_Course_AT_PoliMi")

In [3]:
!pip install "numpy<1.24.0"

In [4]:
import numpy
print(numpy.__version__)

1.23.5


In [5]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [6]:
urm_path = '/kaggle/input/recsyscourse/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

In [7]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

In [25]:
save_npz("./URM_train.npz", URM_train)
save_npz("./URM_test.npz", URM_test)
save_npz("./URM_validation.npz", URM_validation)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout

cutoff_list=[5, 10, 15]

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

EvaluatorHoldout: Ignoring 2938 (22.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2524 (19.4%) Users that have less than 1 test interactions


# Hybrid of models with the same structure

The underlying idea is that if you have some models with the same underlying structure, you can merge them in a single one. The result is to define a function that give the various model's parameters creates a new model with its own parameters.
In this example we will see how to combine different item-item similarity models in a new one.

In particular: `ItemKNNCFRecommender` and `P3AlphaRecommender`. Both use and learn a similarity matrix to provide recommendations. Notice that here we are merging a KNN-based with a Graph-based model.


Let's start using our best model yet

SLIM elasticnet with removed empty profiles and the hyperparameters: 
alpha = 0.0015746723778813712, l1_ratio = 0.005,topK = 100, max_iter=100, tol=1e-4,positive_only=True. On validation it had MAP@10 on val is 0.08221

---

# SLIM ElasticNet

In [9]:
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Incremental_Similarity_Builder
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import sys
import time


class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available

    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.

        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)
        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        similarity_builder = Incremental_Similarity_Builder(self.n_items, initial_data_block=self.n_items*self.topK, dtype = np.float32)

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values
            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            # Check if there are more data points than topK, if so, extract the set of K best values
            if len(nonzero_model_coef_value) > self.topK:
                # Partition the data because this operation does not require to fully sort the data
                relevant_items_partition = np.argpartition(-np.abs(nonzero_model_coef_value), self.topK-1, axis=0)[0:self.topK]
                nonzero_model_coef_index = nonzero_model_coef_index[relevant_items_partition]
                nonzero_model_coef_value = nonzero_model_coef_value[relevant_items_partition]

            similarity_builder.add_data_lists(row_list_to_add=nonzero_model_coef_index,
                                              col_list_to_add=np.ones(len(nonzero_model_coef_index), dtype = int) * currentItem,
                                              data_list_to_add=nonzero_model_coef_value)


            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        self.W_sparse = similarity_builder.get_SparseMatrix()

In [10]:
slim_elastic_recommender = SLIMElasticNetRecommender(URM_train)
slim_elastic_recommender.fit(alpha = 0.0015746723778813712, l1_ratio = 0.005,topK = 100)

SLIMElasticNetRecommender: URM Detected 850 ( 6.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 468 ( 2.1%) items with no interactions.


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

SLIMElasticNetRecommender: Processed 9668 (43.3%) in 5.00 min. Items per second: 32.22


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

SLIMElasticNetRecommender: Processed 18967 (84.9%) in 10.00 min. Items per second: 31.61


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

SLIMElasticNetRecommender: Processed 22348 (100.0%) in 12.23 min. Items per second: 30.46


In [12]:
slim_elastic_recommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2082561 stored elements in Compressed Sparse Row format>

In [27]:
from scipy.sparse import save_npz
save_npz("./slim_elastic_test.npz", slim_elastic_recommender.W_sparse)

---

## EASE_R

In [15]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [17]:
easer_recommender = EASE_R_Recommender(URM_train)
easer_recommender.fit(topK=227,l2_norm=56.530583)

EASE_R_Recommender: URM Detected 850 ( 6.5%) users with no interactions.
EASE_R_Recommender: URM Detected 468 ( 2.1%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.00 min


In [22]:
sps.csr_matrix(easer_recommender.W_sparse)

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 4965398 stored elements in Compressed Sparse Row format>

In [28]:
save_npz("./easer_test.npz", sps.csr_matrix(easer_recommender.W_sparse))

---

# IBCF 

Item-base CF where hyperparamters were obtained with 10-fold CV.
topK = 13,shrink=10, silimarity = 'cosine'

In [55]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [56]:
IBCFrecommender = ItemKNNCFRecommender(URM_train)
IBCFrecommender.fit(topK=14,shrink=7)

ItemKNNCFRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 1780.97 column/sec. Elapsed time 12.55 sec


In [57]:
IBCFrecommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 305225 stored elements in Compressed Sparse Row format>

In [58]:
save_npz("./IBCF_test.npz", IBCFrecommender.W_sparse)

---

# Evaluation

In [29]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_model(URM_test, recommender_object, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, cutoff=at,remove_seen_flag=True)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    mean_precision = cumulative_precision / num_eval
    mean_recall = cumulative_recall / num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP))
    return MAP, mean_precision, mean_recall

In [30]:
evaluate_model(URM_test,slim_elastic_recommender)

Recommender results are: Precision = 811.5000, Recall = 1150.4697, MAP = 0.0656


(0.06560423622146486, 0.07727835444243808, 0.10955811000577838)

In [31]:
evaluate_model(URM_test,easer_recommender)

Recommender results are: Precision = 792.0000, Recall = 1203.0095, MAP = 0.0642


(0.06423663887161984, 0.07542138843920039, 0.11456141835006804)

---

Now that we have both models fit with `URM_train` how can we merge them?


Well, we could take the advantage that they have the same underlying structure, i.e. a similarity matrix

In [24]:
slim_elastic_recommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2085811 stored elements in Compressed Sparse Row format>

In [25]:
IBCFrecommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 305280 stored elements in Compressed Sparse Row format>

When we create hybrids we always have to account for:
* A weight, which is a hyperparameter to tune. 
* What is the range and distribution of the values we are merging

Imagine that the values of one similarity are orders of magnitude higher than the other, then the smaller values would get absorbed by the bigger ones. 

In [44]:
alpha = 0.2
new_similarity = (1 - alpha) * slim_elastic_recommender.W_sparse + alpha * IBCFrecommender.W_sparse
new_similarity

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2115592 stored elements in Compressed Sparse Row format>

Moreover, we can apply the same optimizations to this similarity as we applied to the others (for example: topK neighbors)

The framework includes a recommender class which supports a custom similarity

In [45]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 836 ( 6.4%) users with no interactions.

ItemKNNCustomSimilarityRecommender: URM Detected 476 ( 2.1%) items with no interactions.


In [46]:
evaluate_model(URM_test,recommender_object)

Recommender results are: Precision = 833.3000, Recall = 1220.9303, MAP = 0.0677


(0.06770425768651123, 0.07962732919255107, 0.11666796467752119)

# Find the model using our best 4 submissions

In [1]:
from scipy.sparse import load_npz

Best submission is SLIM_elastic:

In [2]:
SLIM_elastic_sim = load_npz("./content/item_item_similarity/slim_elastic_complete.npz")

In [3]:
SLIM_elastic_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2197697 stored elements in Compressed Sparse Row format>

Then we have:

In [4]:
easer_sim = load_npz("./content/item_item_similarity/easer_complete.npz")

In [5]:
IBCF_sim = load_npz("./content/item_item_similarity/IBCF_complete.npz")

In [6]:
IBCF_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 310959 stored elements in Compressed Sparse Row format>

In [8]:
rp3beta_sim = load_npz("./content/item_item_similarity/rp3beta_complete.npz")

In [9]:
rp3beta_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 3619038 stored elements in Compressed Sparse Row format>

## Trying merging them

In [17]:
URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

In [18]:
URM_all = URM_all.tocsr()

In [39]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

Trying with SLIM and easer

In [36]:
S_slim_elastic = slim_elastic_recommender.W_sparse

In [37]:
S_slim_elastic

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2082561 stored elements in Compressed Sparse Row format>

In [32]:
S_easer = sps.csr_matrix(easer_recommender.W_sparse)

In [33]:
S_easer

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 4965398 stored elements in Compressed Sparse Row format>

In [41]:
alpha = 0
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 811.5000, Recall = 1150.4697, MAP = 0.0656


(0.06560423622146486, 0.07727835444243808, 0.10955811000577838)

In [43]:
alpha = 0.2
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 823.7000, Recall = 1191.0876, MAP = 0.0668


(0.06676119638055801, 0.0784401485572846, 0.11342611141887478)

In [44]:
alpha = 0.4
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 832.6000, Recall = 1213.3870, MAP = 0.0673


(0.06731588968035258, 0.07928768688696755, 0.11554966625371846)

In [45]:
alpha = 0.6
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 829.5000, Recall = 1230.0016, MAP = 0.0672


(0.06716932989429121, 0.07899247690696555, 0.11713185295960744)

In [46]:
alpha = 0.8
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 818.8000, Recall = 1230.9644, MAP = 0.0662


(0.06617982080867686, 0.07797352633082989, 0.11722353790497658)

The best is 0.4<alpha<0.6

In [47]:
alpha = 0.55
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 830.5000, Recall = 1228.7009, MAP = 0.0672


(0.06723034456651912, 0.07908770593277267, 0.11700798853837799)

In [49]:
alpha = 0.45
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 833.1000, Recall = 1219.3309, MAP = 0.0674


(0.06738276262319025, 0.0793353013998711, 0.1161156975964578)

In [50]:
alpha = 0.5
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 834.0000, Recall = 1226.7931, MAP = 0.0673


(0.06733403181148498, 0.07942100752309748, 0.11682630926996648)

We got the best at alpha = 0.45

In [51]:
alpha = 0.43
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 832.9000, Recall = 1217.9704, MAP = 0.0674


(0.06737265490126312, 0.07931625559470969, 0.1159861331341172)

In [52]:
alpha = 0.47
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 834.1000, Recall = 1225.4241, MAP = 0.0673


(0.06734576550784521, 0.07943053042567817, 0.11669594125961753)

0.45 better than 0.43

In [53]:
alpha = 0.44
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 832.5000, Recall = 1217.5780, MAP = 0.0674


(0.06737810770172468, 0.07927816398438685, 0.11594877026164925)

In [54]:
alpha = 0.46
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 833.6000, Recall = 1221.7811, MAP = 0.0673


(0.0673453345749095, 0.07938291591277463, 0.11634902415695238)

The best one is with alpha = 0.45.

---

Now let's try using IBCF

In [60]:
S_IBCF = IBCFrecommender.W_sparse

In [62]:
S_IBCF

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 305225 stored elements in Compressed Sparse Row format>

Now, we have the following map values:
- SLIM_Elastic: 0.0656
- IBCF: 0.0593
- SLIM_Elastic + Easer: 0.0674

In [63]:
alpha = 0.3
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 831.0000, Recall = 1216.7203, MAP = 0.0669


(0.06691038165293431, 0.07913532044567616, 0.11586709094269772)

In [65]:
alpha = 0.5
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 801.1000, Recall = 1190.0371, MAP = 0.0641


(0.06414257838981556, 0.07628797257404467, 0.11332607026726237)

So alpha < 0.5

In [66]:
alpha = 0.1
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 837.2000, Recall = 1209.3347, MAP = 0.0680


(0.06800828684470134, 0.07972574040568009, 0.11516376595899146)

In [68]:
alpha = 0.2
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 840.6000, Recall = 1220.7297, MAP = 0.0675


(0.06749119356447561, 0.08004951909342417, 0.11624889783231607)

So alpha < 0.2

In [70]:
alpha = 0.08
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 835.3000, Recall = 1210.0707, MAP = 0.0679


(0.06791749801618177, 0.0795448052566466, 0.11523385613826373)

In [71]:
alpha = 0.15
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 838.6000, Recall = 1216.4414, MAP = 0.0679


(0.06785192108124204, 0.07985906104181006, 0.11584052794152432)

0.1 < alpha < 0.15

In [72]:
alpha = 0.13
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 836.6000, Recall = 1211.4849, MAP = 0.0679


(0.06789061916477766, 0.07966860299019585, 0.11536853136344688)

In [73]:
alpha = 0.12
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 838.4000, Recall = 1214.4074, MAP = 0.0681


(0.06806266468185265, 0.07984001523664858, 0.11564683541515483)

In [74]:
alpha = 0.11
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 838.2000, Recall = 1210.6776, MAP = 0.0680


(0.06799134015995147, 0.07982096943148709, 0.11529165266140842)

In [76]:
alpha = 0.125
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 837.0000, Recall = 1212.5920, MAP = 0.0680


(0.06796234422322449, 0.0797066946005187, 0.11547395202145841)

We got alpha = 0.12.

---

Now, we have the following map values:
- SLIM_Elastic: 0.0656
- IBCF: 0.0593
- SLIM_Elastic + Easer (alpha = 0.45) : 0.06738276262319025
- SLIM_Elastic + IBCF (alpha = 0.12) : 0.06806266468185265

Let's try to merge the 3 best models

In [77]:
alpha = 0.35
beta = 0.08
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 845.8000, Recall = 1241.8775, MAP = 0.0684


(0.06839468776432236, 0.08054471002762097, 0.11826278369883793)

We just got an high score!

In [78]:
alpha = 0.4
beta = 0.08
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.3000, Recall = 1249.4665, MAP = 0.0684


(0.06839884175268014, 0.08059232454052448, 0.11898548229529471)

In [79]:
alpha = 0.4
beta = 0.1
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.7000, Recall = 1250.7387, MAP = 0.0683


(0.06830202946034279, 0.08063041615084732, 0.11910663122812372)

In [80]:
alpha = 0.35
beta = 0.1
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 847.1000, Recall = 1247.7045, MAP = 0.0684


(0.06841077492253815, 0.08066850776117004, 0.11881768527745744)

In [81]:
alpha = 0.3
beta = 0.12
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.9000, Recall = 1243.5436, MAP = 0.0684


(0.06844195209641683, 0.0806494619560087, 0.1184214494159486)

In [84]:
alpha = 0.3
beta = 0.11
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.4000, Recall = 1241.9562, MAP = 0.0684


(0.06840833208448684, 0.08060184744310511, 0.11827028068255935)

Chaning beta is not useful. We got 0.06844195209641683 with alpha = 0.3 and beta = 0.11

In [85]:
alpha = 0.28
beta = 0.11
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 844.8000, Recall = 1238.6806, MAP = 0.0684


(0.06843698145438097, 0.08044948100181386, 0.11795834983881517)

In [86]:
alpha = 0.25
beta = 0.11
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 845.9000, Recall = 1237.5750, MAP = 0.0683


(0.06832770977623938, 0.08055423293020163, 0.1178530614741371)

In [87]:
alpha = 0.25
beta = 0.13
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.3000, Recall = 1238.3872, MAP = 0.0683


(0.06827242186187303, 0.08059232454052444, 0.11793041091508692)

In [88]:
alpha = 0.27
beta = 0.12
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.7000, Recall = 1240.5324, MAP = 0.0684


(0.06837358574159705, 0.08063041615084728, 0.11813469225109782)

Diminshing alpha and increasing beta isn't successful. Best: alpha = 0.28, beta = 0.11, MAP = 0.06843698145438097

In [89]:
alpha = 0.28
beta = 0.12
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.5000, Recall = 1239.9348, MAP = 0.0684


(0.06838441693359698, 0.08061137034568587, 0.11807777931851046)

In [90]:
alpha = 0.28
beta = 0.1
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 844.8000, Recall = 1236.7917, MAP = 0.0684


(0.06840299319176023, 0.08044948100181386, 0.11777847017825283)

Neither only changing beta.

In [91]:
alpha = 0.27
beta = 0.11
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 844.8000, Recall = 1236.9653, MAP = 0.0684


(0.06835454406026745, 0.08044948100181384, 0.11779499892879446)

In [92]:
alpha = 0.29
beta = 0.12
new_similarity = (1 - alpha-beta) * S_slim_elastic + alpha * S_easer + beta * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 846.0000, Recall = 1241.5276, MAP = 0.0684


(0.06839594490323034, 0.08056375583278232, 0.11822946275628667)

Final best values: alpha = 0.28, beta = 0.11, MAP = 0.06843698145438097

---

Now let's trying SLIM_elasticnet + rp3beta.

In [93]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [94]:
# Create an instance of the P3alphaRecommender class
recommender = RP3betaRecommender(URM_train)

# Fit the recommender (you can customize the fit parameters if needed)
recommender.fit(topK=190,alpha=0.3, beta=0.4233057604632479, min_rating=0.15, implicit=True, normalize_similarity=False)

RP3betaRecommender: URM Detected 850 ( 6.5%) users with no interactions.
RP3betaRecommender: URM Detected 468 ( 2.1%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 1451.32 column/sec. Elapsed time 15.40 sec


In [95]:
from scipy.sparse import save_npz
save_npz("./rp3beta_test.npz", recommender.W_sparse)

In [96]:
S_rp3beta = recommender.W_sparse
S_rp3beta

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 3298632 stored elements in Compressed Sparse Row format>

In [98]:
evaluate_model(URM_test, recommender)

Recommender results are: Precision = 694.9000, Recall = 1095.2549, MAP = 0.0562


(0.056197075656137956, 0.06617465003333382, 0.10430005374807477)

So, till now, we have:
- SLIM_Elastic: 0.0656
- IBCF: 0.0593
- SLIM_Elastic + Easer (alpha = 0.45) : 0.06738276262319025
- SLIM_Elastic + IBCF (alpha = 0.12) : 0.06806266468185265
- SLIM, Easer, IBCF with alpha = 0.28, beta = 0.11: MAP = 0.06843698145438097

In [99]:
alpha = 0.1
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 827.9000, Recall = 1245.0629, MAP = 0.0675


(0.06749367953031074, 0.07884011046567424, 0.1185661258233699)

In [100]:
alpha = 0.3
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 764.2000, Recall = 1195.0910, MAP = 0.0632


(0.06315531763234865, 0.07277402152176403, 0.1138073469849079)

In [101]:
alpha = 0.2
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)
evaluate_model(URM_validation,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 794.5000, Recall = 1222.9193, MAP = 0.0654
Recommender results are: Precision = 631.2000, Recall = 1174.3091, MAP = 0.0597


(0.05965477766897385, 0.06257559234658804, 0.11641807447277196)

In [102]:
alpha = 0.05
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 834.0000, Recall = 1222.2121, MAP = 0.0676


(0.06761360784989456, 0.07942100752309736, 0.11639006638051703)

In [103]:
alpha = 0.15
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 813.9000, Recall = 1238.5808, MAP = 0.0666


(0.06660019103628771, 0.07750690410437529, 0.11794884496474052)

In [104]:
alpha = 0.03
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 828.1000, Recall = 1201.4127, MAP = 0.0672


(0.06718921115780435, 0.07885915627083567, 0.11440935756647046)

We got 0.05 < alpha < 0.1 (let's say 0.07) with map = 0.0676

In [105]:
evaluate_model(URM_test,recommender_object)

Recommender results are: Precision = 828.1000, Recall = 1201.4127, MAP = 0.0672


(0.06718921115780435, 0.07885915627083567, 0.11440935756647046)

In [106]:
evaluate_model(URM_validation,recommender_object)

Recommender results are: Precision = 653.9000, Recall = 1158.9958, MAP = 0.0605


(0.06052784696359119, 0.06482601368097904, 0.11489995454943401)

In [107]:
alpha = 0
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 811.5000, Recall = 1150.4697, MAP = 0.0656


(0.06560423622146486, 0.07727835444243808, 0.10955811000577838)

In [108]:
evaluate_model(URM_validation,recommender_object)

Recommender results are: Precision = 637.6000, Recall = 1108.4401, MAP = 0.0592


(0.05918934817978016, 0.06321007237038105, 0.10988798460209802)

---

So, till now, we have:
- SLIM_Elastic: 0.0656
- IBCF: 0.0593
- SLIM_Elastic + Easer (alpha = 0.45) : 0.06738276262319025
- SLIM_Elastic + IBCF (alpha = 0.12) : 0.06806266468185265
- SLIM, Easer, IBCF with alpha = 0.28, beta = 0.11: MAP = 0.06843698145438097
- SLIM + rp3beta (alpha = 0.07 ): MAP = 0.0676

Try Easer + IBCF

In [109]:
alpha = 0.1
new_similarity = (1 - alpha) * S_easer + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 783.4000, Recall = 1220.3371, MAP = 0.0638


(0.06379458875996613, 0.07460241881725971, 0.11621151378933754)

In [112]:
alpha = 0.3
new_similarity = (1 - alpha) * S_easer + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 746.0000, Recall = 1168.6804, MAP = 0.0609


(0.0608590400417538, 0.07104085325207543, 0.11129229451521666)

We got less than SLIM+rp3beta -> abort

Easer + rp3beta

In [110]:
alpha = 0.1
new_similarity = (1 - alpha) * S_easer + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 783.4000, Recall = 1220.3371, MAP = 0.0638


(0.06379458875996613, 0.07460241881725971, 0.11621151378933754)

In [111]:
alpha = 0.3
new_similarity = (1 - alpha) * S_easer + alpha * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 746.0000, Recall = 1168.6804, MAP = 0.0609


(0.0608590400417538, 0.07104085325207543, 0.11129229451521666)

Same for this

---

Last try: merge all the 4 models.

So, till now, we have:
- SLIM_Elastic: 0.0656
- IBCF: 0.0593
- SLIM_Elastic + Easer (alpha = 0.45) : 0.06738276262319025
- SLIM_Elastic + IBCF (alpha = 0.12) : 0.06806266468185265
- SLIM, Easer, IBCF with alpha = 0.28, beta = 0.11: MAP = 0.06843698145438097
- SLIM + rp3beta (alpha = 0.07 ): MAP = 0.0676

In [113]:
alpha = 0.27
beta = 0.1
gamma = 0.06
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 850.3000, Recall = 1268.1150, MAP = 0.0685


(0.06845885434875235, 0.08097324064375266, 0.12076135982419689)

Just got better than our best merged recommender!

In [116]:
alpha = 0.3
beta = 0.1
gamma = 0.06
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 847.5000, Recall = 1266.5085, MAP = 0.0682


(0.06824011234373786, 0.08070659937149287, 0.12060837429475432)

In [115]:
alpha = 0.25
beta = 0.1
gamma = 0.06
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 851.4000, Recall = 1268.0124, MAP = 0.0685


(0.06852611776598577, 0.08107799257214043, 0.12075158486280486)

In [117]:
alpha = 0.23
beta = 0.1
gamma = 0.08
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 842.0000, Recall = 1260.4498, MAP = 0.0679


(0.06786739453829253, 0.08018283972955395, 0.12003140659959724)

In [118]:
alpha = 0.25
beta = 0.1
gamma = 0.08
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 841.8000, Recall = 1259.5925, MAP = 0.0678


(0.06780255086565193, 0.0801637939243925, 0.11994977101064935)

In [119]:
alpha = 0.25
beta = 0.1
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 852.6000, Recall = 1269.6929, MAP = 0.0686


(0.06856603975784203, 0.081192267403109, 0.12091161383257475)

In [120]:
alpha = 0.25
beta = 0.05
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 851.9000, Recall = 1250.6696, MAP = 0.0686


(0.0686138259721102, 0.08112560708504404, 0.11910004669792373)

In [121]:
alpha = 0.28
beta = 0.05
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 852.9000, Recall = 1256.7105, MAP = 0.0687


(0.06866984500438324, 0.08122083611085114, 0.1196753208774909)

In [122]:
alpha = 0.29
beta = 0.05
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.0000, Recall = 1259.8846, MAP = 0.0687


(0.06868477888427207, 0.08123035901343181, 0.11997757883641151)

In [123]:
alpha = 0.29
beta = 0.04
gamma = 0.02
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 849.1000, Recall = 1246.0844, MAP = 0.0684


(0.0684267652231898, 0.08085896581278422, 0.1186634070250744)

In [125]:
alpha = 0.3
beta = 0.04
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 849.7000, Recall = 1255.0958, MAP = 0.0686


(0.06858773852153716, 0.08091610322826848, 0.11952154959361774)

In [126]:
alpha = 0.29
beta = 0.05
gamma = 0.02
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 849.9000, Recall = 1247.6822, MAP = 0.0685


(0.06846454027318023, 0.08093514903342992, 0.11881555732947029)

For now our best model is with alpha = 0.29, beta = 0.05, gamma = 0.03. I want to increase beta (a lot)

In [127]:
alpha = 0.2
beta = 0.3
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 832.8000, Recall = 1241.3676, MAP = 0.0667


(0.06674402583921679, 0.07930673269212886, 0.1182142241240132)

In [128]:
alpha = 0.1
beta = 0.3
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 835.7000, Recall = 1234.9573, MAP = 0.0670


(0.06696683826126137, 0.07958289686696936, 0.11760377894158285)

In [129]:
alpha = 0.2
beta = 0.1
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.1000, Recall = 1265.6817, MAP = 0.0687


(0.06868167780767077, 0.08123988191601249, 0.12052963813415704)

We got 0.6868 again

In [130]:
alpha = 0.2
beta = 0.13
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.7000, Recall = 1269.8208, MAP = 0.0686


(0.06858397161853491, 0.08129701933149674, 0.1209237958160724)

In [131]:
alpha = 0.23
beta = 0.11
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.6000, Recall = 1268.9373, MAP = 0.0687


(0.06866430020498089, 0.08128749642891607, 0.12083966264907624)

In [132]:
alpha = 0.21
beta = 0.1
gamma = 0.04
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.6000, Recall = 1266.0251, MAP = 0.0687


(0.06867950714252447, 0.08128749642891607, 0.12056233790466694)

In [133]:
alpha = 0.21
beta = 0.1
gamma = 0.06
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.3000, Recall = 1271.2120, MAP = 0.0686


(0.06859715453439483, 0.08125892772117382, 0.12105628037982072)

In [135]:
alpha = 0.2
beta = 0.1
gamma = 0.05
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.1000, Recall = 1265.6817, MAP = 0.0687


(0.06868167780767077, 0.08123988191601249, 0.12052963813415704)

So now we have 2 good models:
alpha = 0.2
beta = 0.1
gamma = 0.05

and

alpha = 0.29
beta = 0.05
gamma = 0.03

Let's see on the validation set to choose among them

alpha = 0.2
beta = 0.1
gamma = 0.05

In [136]:
evaluate_model(URM_validation,recommender_object)

Recommender results are: Precision = 674.3000, Recall = 1219.4995, MAP = 0.0620


(0.061970120002588634, 0.06684841875681946, 0.1208981383365634)

alpha = 0.29
beta = 0.05
gamma = 0.03

In [137]:
alpha = 0.29
beta = 0.05
gamma = 0.03
new_similarity = (1 - alpha-beta-gamma) * S_slim_elastic + alpha * S_easer + beta * S_IBCF + gamma * S_rp3beta
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_test,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 850 ( 6.5%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 468 ( 2.1%) items with no interactions.
Recommender results are: Precision = 853.0000, Recall = 1259.8846, MAP = 0.0687


(0.06868477888427207, 0.08123035901343181, 0.11997757883641151)

In [138]:
evaluate_model(URM_validation,recommender_object)

Recommender results are: Precision = 672.2000, Recall = 1212.2994, MAP = 0.0618


(0.061768146322009734, 0.06664022999901227, 0.12018433994606843)

Best model: alpha = 0.2
beta = 0.1
gamma = 0.05